# User Interface Configuration and Information
We want to create a small user interface for our prototype in emoji prediction

## Needed
We want to define needed components for this UI

In [1]:
import random
import ipywidgets as widgets
from IPython.display import display, clear_output
import math
import datetime

### Trigger refresh of prediction
each action of typing and sending should yield a new updated prediction for best fitting emojis

Initial definition of emojis used later

In [2]:
#locally defined based on the first analysis of parts of our twitter data: resulting in the 20 most used emojis
#we used them for our first approaches of prediction

top_emojis = list("😳😋😀😌😏😔😒😎😢😅😁😉🙌🙏😘😊😩😍😭😂")
#possible initial set of predictions, only used in naive test cases
predictions = ["🤐","🤑","🤒","🤓","🤔","🤕","🤗","🤘"]

#### Advanced Approach
define the classifier for advanced prediction, used for the sentiment prediction

In [3]:
#navigation into right path and generating classifier
import sys
sys.path.append("..")
sys.path.append("../naive_approach")

import advanced_approach.twitter_learning as twl
#clf_advanced = stl.pipeline_manager.load_pipeline_from_files( '"/Users/Carsten/DataSets/NLP_LAB/tfidf_final/final_epoch01.pipeline', ['keras_model'], ['vectorizer', 'keras_model'])
clf_advanced = twl.pipeline_manager.load_from_pipeline_file("/home/jonas/Dokumente/NLP_DATA/python_dumps/pipelines/tfidf_final/final_epoch01.pipeline")
import Tools.Emoji_Distance as ed

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /home/jonas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jonas/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/jonas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### Generate new Sample for online learning / reinforcement learning

In [4]:
def generate_new_training_sample (msg, emoji):
    sentiment = ed.emoji_to_sentiment_vector(emoji)
    
    #TODO message msg could be filtred
    text = msg
    return text, sentiment

#### Naive Approach
for topic related emoji prediction

In [5]:
#sys.path.append("..")
#print(sys.path)

import naive_approach as clf_naive

In [6]:
tmp_dict = clf_naive.prepareData()

#### Merge Predictions
combine the predictions of both approaches

In [7]:
def merged_prediction(msg , split = 0.5 , number = 8, target_emojis = top_emojis):
    
    #calc ratio of prediction splitted between advanced aprroach and naive approach
    number_advanced = round(split*number)
    number_naive = round((1-split)*number)
    
    #predict emojis with the naive approach
    prediction_naive , prediction_naive_values = clf_naive.predict(sentence = msg, lookup= tmp_dict, n = number_naive)

    #filter 0 values
    tmp1 = []
    tmp2 = []
    epsilon = 0.0001

    for i in range(len(prediction_naive)):
        if(abs(prediction_naive_values[i]) > epsilon):
            tmp1.append(prediction_naive[i])
            tmp2.append(prediction_naive[i])

    prediction_naive = tmp1
    prediction_naive_values = tmp2
    
    if(len(prediction_naive) < number_naive):
        #print("only few matches")
        number_advanced = number - len(prediction_naive)
        
    #print(number, number_advanced, number_naive)
    
    #predict the advanced approach
    sentiment = clf_advanced.predict([msg])
    prediction_advanced = ed.sentiment_vector_to_emoji(sentiment,n_results = number_advanced, custom_target_emojis=target_emojis)
        
    #concat both predictions
    prediction = list(prediction_advanced)+list(prediction_naive)
    
    return prediction[:number], len(tmp1)

Actions triggered when something is changed

In [8]:
def trigger_new_prediction(all_chat, current_message):
    global predictions
    
    #random prediction for  initial test
    #random.shuffle(predictions)
    
    #first prediction only using advanced approach
    #sent = clf_advanced.predict([current_message])
    #p = ed.sentiment_vector_to_emoji(sent,n_results = 8, custom_target_emojis=top_emojis)
    
    #merged prediction
    if(current_message != ""):
        p,n = merged_prediction(msg = current_message, target_emojis=top_emojis)

        predictions = p
        update_descriptions()

### User Output
the wiritten text as an overview or list of text

In [9]:
all_text = "no text yet \n"

out = widgets.Output(layout = widgets.Layout(max_height = "500px"))

### User Input
the user has to interact with our UI so hee needs:

#### Text Input field
a simple line for text input on the bottom of UI

In [10]:
text_input = widgets.Text()

In [11]:
def submit_new_message(p):
    global all_text
    bar = "----------- \n"
    time = str(datetime.datetime.now())+"\n"
    msg = text_input.value +"\n"
    new_message = bar + time + msg
    all_text += new_message 
    
    with out:
        clear_output()
        print(all_text)
        
    trigger_new_prediction(all_text, text_input.value)
    update_descriptions()
    text_input.value = ""

In [12]:
text_input.on_submit(submit_new_message)

#text_input.observe(lambda b: trigger_new_prediction(None, text_input.value))

#### A sent button to enter yout typed in message
alternatively it should be also possible to simply type enter

In [13]:
sent_button = widgets.Button(description = "Sent")

sent_button.on_click(submit_new_message)

#### A list of buttons for selecting predicted emojis
a set of fixed size of buttons with a dynamic changeable labeling replaced by the unicode emoji

In [14]:
p0_button = widgets.Button(description = "p0")

def on_p0_button_click(p):
    update_descriptions()    
    #with out:
    text_input.value += " "+predictions[0]

p0_button.on_click(on_p0_button_click)

In [15]:
p1_button = widgets.Button(description = "p1")

def on_p1_button_click(p):
    update_descriptions()
    with out:
        text_input.value += " "+predictions[1]

p1_button.on_click(on_p1_button_click)

In [16]:
p2_button = widgets.Button(description = "p2")

def on_p2_button_click(p):
    update_descriptions()
    with out:
        text_input.value += " "+predictions[2]

p2_button.on_click(on_p2_button_click)

In [17]:
p3_button = widgets.Button(description = "p3")

def on_p3_button_click(p):
    update_descriptions()
    with out:
        text_input.value += " "+predictions[3]

p3_button.on_click(on_p3_button_click)

In [18]:
p4_button = widgets.Button(description = "p4")

def on_p4_button_click(p):
    update_descriptions()
    with out:
        text_input.value += " "+predictions[4]

p4_button.on_click(on_p4_button_click)

In [19]:
p5_button = widgets.Button(description = "p5")

def on_p5_button_click(p):
    update_descriptions()
    with out:
        text_input.value += " "+predictions[5]

p5_button.on_click(on_p5_button_click)

In [20]:
p6_button = widgets.Button(description = "p6")

def on_p6_button_click(p):
    update_descriptions()
    with out:
        text_input.value += " "+predictions[6]

p6_button.on_click(on_p6_button_click)

In [21]:
p7_button = widgets.Button(description = "p7")

def on_p7_button_click(p):
    update_descriptions()
    with out:
        text_input.value += " "+predictions[7]

p7_button.on_click(on_p7_button_click)

In [22]:
def update_descriptions():
    global predictions
    p0_button.description = predictions[0]
    p1_button.description = predictions[1]
    p2_button.description = predictions[2]
    p3_button.description = predictions[3]
    p4_button.description = predictions[4]
    p5_button.description = predictions[5]
    p6_button.description = predictions[6]
    p7_button.description = predictions[7]

### Additional
Developer Information
#### Output of in and out commands
#### Prob distribution or whole list of sorted emojis
#### configuration information

## Prototype UI

In [23]:
text_output = widgets.VBox([out])
all_prediction_buttons = widgets.HBox([p0_button,p1_button,p2_button,p3_button,p4_button,p5_button,p6_button,p7_button])
user_input = widgets.HBox([text_input,sent_button])
total_layout = widgets.VBox([text_output,all_prediction_buttons,user_input],layout = widgets.Layout(max_width = "450px"))
display(total_layout)

improve naive approach
word2vec

longer sentences
own training set

## I Am Groot

In [24]:
text_input_groot = widgets.Text()
out_groot = widgets.Output(layout = widgets.Layout(max_height = "500px"))
all_text_groot = "no text yet \n"
text_output_groot = widgets.VBox([out_groot])
user_input_groot = widgets.HBox([text_input_groot])
total_layout_groot = widgets.VBox([text_output_groot,user_input_groot],layout = widgets.Layout(max_width = "450px"))

In [25]:
def submit_new_message_groot(p):
    global all_text_groot
    bar = "----------- \n"
    time = str(datetime.datetime.now())+"\n"
    msg = "User: "+text_input_groot.value +"\n"
    new_message = bar + time + msg
    all_text_groot += new_message 
    
    mp, n_topic = merged_prediction(msg)
    
    answer = "ChatGROOT: I AM GROOT" + mp[0]
    
    if n_topic > 0:
        answer += mp[8 - n_topic]
    
    all_text_groot += answer + "\n"
    
    with out_groot:
        clear_output()
        print(all_text_groot)
        
    text_input_groot.value = ""

In [26]:
text_input_groot.on_submit(submit_new_message_groot)
display(total_layout_groot)